In [1]:
import sys
fbxpath = r'D:\Software\fbx_python37_x64'
if fbxpath not in sys.path:
    sys.path.append(fbxpath)

3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

import moderngl
import numpy as np
from pyrr import Matrix44


from IPython.display import clear_output, display, Image
import ipywidgets as widgets

import PIL
print(PIL)

def grid(size, steps):
    u = np.repeat(np.linspace(-size, size, steps), 2)
    v = np.tile([-size, size], steps)
    w = np.zeros(steps * 2)
    return np.concatenate([np.dstack([u, w, v]), np.dstack([v, w, u])])

# -------------------
# CREATE CONTEXT HERE
# -------------------

ctx = moderngl.create_standalone_context()

prog = ctx.program(
    vertex_shader='''
        #version 330
        uniform mat4 Mvp;
        in vec3 in_vert;
        void main() {
            gl_Position = Mvp * vec4(in_vert, 1.0);
        }
    ''',
    fragment_shader='''
        #version 330
        out vec4 f_color;
        void main() {
            f_color = vec4(0.1, 0.1, 0.1, 1.0);
        }
    ''',
)



vbo = ctx.buffer(grid(15, 10).astype('f4'))
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert')

fbo = ctx.simple_framebuffer([1280//2,720//2])
fbo.use()



def _render(x):
    ctx.clear(1.0, 1.0, 1.0)
    ctx.enable(moderngl.DEPTH_TEST)
    
    proj = Matrix44.perspective_projection(45.0, fbo.size[1]/fbo.size[0], 0.1, 1000.0)
    lookat = Matrix44.look_at(
        (0,x['new'],10),
        (0.0, 0.0, 0.0),
        (0.0, 1.0, 0.0),
    )

    prog['Mvp'].write((proj * lookat).astype('f4'))
    vao.render(moderngl.LINES)

    data = fbo.read(components=3)
    img = np.frombuffer(data, np.dtype(np.uint8)).reshape(fbo.size[1],fbo.size[0],3)
    with output:
        clear_output()
        display(PIL.Image.fromarray(img))

    

'''
widgets.interact(
    _render, 
    x=widgets.IntSlider(
        min=0, 
        max=20, 
        step=1, 
        value=10, 
        layout=widgets.Layout(height='auto', width='auto')
    )
)
'''

slider = widgets.IntSlider(
        min=0, 
        max=20, 
        step=1, 
        value=10, 
        layout=widgets.Layout(height='auto', width='auto')
    )
output = widgets.Output()
display(slider, output)
#_render()

slider.observe(_render, names='value')

<module 'PIL' from 'd:\\software\\python37_envs\\torch\\lib\\site-packages\\PIL\\__init__.py'>


IntSlider(value=10, layout=Layout(height='auto', width='auto'), max=20)

Output()

In [99]:
%%javascript
define('mywidget', ['@jupyter-widgets/base'], function(widgets) {
    var MyWidgetView = widgets.DOMWidgetView.extend({
        render: function() {
            MyWidgetView.__super__.render.apply(this, arguments);
            this._count_changed();
            this.listenTo(this.model, 'change:count', this._count_changed, this);
        },

        _count_changed: function() {
            var old_value = this.model.previous('count');
            var new_value = this.model.get('count');
            this.el.textContent = String(old_value) + ' -> ' + String(new_value);
        }
    });

    return {
        MyWidgetView: MyWidgetView
    }
});

<IPython.core.display.Javascript object>

In [98]:
from ipywidgets import DOMWidget
from traitlets import Unicode, Int

class MyWidget(DOMWidget):
    _view_module = Unicode('mywidget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _view_name = Unicode('MyWidgetView').tag(sync=True)
    count = Int().tag(sync=True)

In [101]:
wid = MyWidget()
display(wid)


MyWidget()

In [102]:
wid.count = 5
